# Seção 4: Classificação binária - base breast cancer
Os dados utilizados neste notebook foram tirados da página [Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29).

## Importação das bibliotecas

In [48]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

## Importação das bases

In [18]:
df_inputs = pd.read_csv("./data/entradas_breast.csv")
df_outputs = pd.read_csv("./data/saidas_breast.csv")

print(f"""- Formato do dataframe df_inputs: {df_inputs.shape}
- Formato do dataframe df_outputs: {df_outputs.shape}""")

- Formato do dataframe df_inputs: (569, 30)
- Formato do dataframe df_outputs: (569, 1)


### Análise das bases importadas:

In [19]:
# df_description = pd.DataFrame()
# for feature in df_inputs.columns:

### Pre-processamento

In [20]:
X_train, X_test, y_train, y_test = train_test_split(df_inputs, df_outputs, test_size=.2)

print(f"""{X_train.shape} e {y_train.shape}
{X_test.shape} e {y_test.shape}""")

(455, 30) e (455, 1)
(114, 30) e (114, 1)


### Modelo

In [39]:
def model_ann():
    """
    Nessa função está a criação da rede neural com a camada de entrada (incluida na adição da primeira camada intermediária),
    três camadas intermediárias e a camada de saída.
    Como trata-se de um problema de classificação binária, a função de ativação usada na camada de saída é a sigmóide.
    """
    model = Sequential()

    # Camada de entrada e intermediária
    model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))

    # Camada intermediária ou oculta
    model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))

    # Camada de saída
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Método de otimização a ser utilizado
    optimizer = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return model

model = KerasClassifier(build_fn=model_ann, epochs=100, batch_size=20, verbose=0)

model.fit(X_train, y_train);

In [40]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred>.5,1,0)

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        44
           1       0.96      0.93      0.94        70

    accuracy                           0.93       114
   macro avg       0.92      0.93      0.93       114
weighted avg       0.93      0.93      0.93       114

[[41  3]
 [ 5 65]]


## Usando validação cruzada

In [41]:
model = KerasClassifier(build_fn=model_ann, epochs=100, batch_size=20)

cross_val = cross_val_score(estimator=model, X=df_inputs, y=df_outputs
                            , cv=2, scoring="f1", verbose=0, n_jobs=-1)

cross_val.mean()

0.8441810344827587

# Overfitting e dropout
Para reduzir o risco de overfitting, pode-se usar o dropout. Esse método zera uma parcelas das entradas de camadas específicas.

In [44]:
def model_ann():
    """
    Nessa função está a criação da rede neural com a camada de entrada (incluida na adição da primeira camada intermediária),
    três camadas intermediárias e a camada de saída.
    Como trata-se de um problema de classificação binária, a função de ativação usada na camada de saída é a sigmóide.
    """
    model = Sequential()

    # Camada de entrada e intermediária
    model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
    model.add(Dropout(0.2))

    # Camada intermediária ou oculta
    model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))

    # Camada de saída
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Método de otimização a ser utilizado
    optimizer = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return model

model = KerasClassifier(build_fn=model_ann, epochs=100, batch_size=20, verbose=0)

model.fit(X_train, y_train);

In [45]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred>.5,1,0)

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.91      0.93        44
           1       0.94      0.97      0.96        70

    accuracy                           0.95       114
   macro avg       0.95      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114

[[40  4]
 [ 2 68]]


## Tuning de hiperparâmetros
Vamos criar uma função baseada em uma criada anteriormente, mas dessa vez generalizaremos alguns dos hiperparâmetros.

In [62]:
def model_ann_tunned(optimizer, loss, kernel_initializer, activation, neurons):
    """
    Nessa função está a criação da rede neural com a camada de entrada (incluida na adição da primeira camada intermediária),
    três camadas intermediárias e a camada de saída.
    Como trata-se de um problema de classificação binária, a função de ativação usada na camada de saída é a sigmóide.
    
    Parâmetros:
        optimizer, loss, kernel_initializer, activation, neurons
    """
    model = Sequential()

    # Camada de entrada e intermediária
    model.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer, input_dim=30))
    model.add(Dropout(0.2))

    # Camada intermediária ou oculta
    model.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer))

    # Camada de saída
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Método de otimização a ser utilizado
    model.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])
    
    return model

model = KerasClassifier(build_fn=model_ann_tunned)

parametros = {'batch_size': [10, 30],
              'epochs': [50, 100],
              'optimizer': ['adam', 'sgd'],
              'loss': ['binary_crossentropy', 'hinge'],
              'kernel_initializer': ['random_uniform', 'normal'],
              'activation': ['relu', 'tanh'],
              'neurons': [16, 8]}

grid_search = GridSearchCV(estimator = model,
                           param_grid = parametros,
                           scoring = 'accuracy',
                           cv = 3)

grid_search = grid_search.fit(X_train, y_train)

melhores_parametros = grid_search.best_params_

melhor_precisao = grid_search.best_score_

Epoch 1/50
31/31 [==============================] - 0s 1ms/step - loss: 1.1410 - binary_accuracy: 0.6205
Epoch 2/50
31/31 [==============================] - 0s 1ms/step - loss: 0.6538 - binary_accuracy: 0.6733
Epoch 3/50
31/31 [==============================] - 0s 1ms/step - loss: 0.6024 - binary_accuracy: 0.6733
Epoch 4/50
31/31 [==============================] - 0s 1ms/step - loss: 0.5169 - binary_accuracy: 0.7129
Epoch 5/50
31/31 [==============================] - 0s 2ms/step - loss: 0.4802 - binary_accuracy: 0.7624
Epoch 6/50
31/31 [==============================] - 0s 2ms/step - loss: 0.5515 - binary_accuracy: 0.6766
Epoch 7/50
31/31 [==============================] - 0s 2ms/step - loss: 0.5093 - binary_accuracy: 0.7195
Epoch 8/50
31/31 [==============================] - 0s 2ms/step - loss: 0.5023 - binary_accuracy: 0.6964
Epoch 9/50
31/31 [==============================] - 0s 2ms/step - loss: 0.5072 - binary_accuracy: 0.7327
Epoch 10/50
31/31 [==============================] - 0s

KeyboardInterrupt: 